<a href="https://colab.research.google.com/github/ljs7463/AnalysisProject/blob/master/%EB%8D%B0%EC%9D%B4%EC%BD%98/analytics/stacking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import lib & load dataset

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os

# 시각화 폰트 설정
if os.name =='posix':
    plt.rc("font", family = "AppleGothic")

else:
    plt.rc("font", family = "Malgun Gothic")

# 경고문자 무시
warnings.filterwarnings(action='ignore')


# 데이터 로드
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_sub = pd.read_csv('sample_submission.csv')
df_info = pd.read_csv('data_info.csv')

# preprocessing & split dataset

In [2]:
## Label Encoding

## train데이터
df_train['preferred_difficulty_level'] = pd.factorize(df_train['preferred_difficulty_level'])[0]
df_train['subscription_type'] = pd.factorize(df_train['subscription_type'])[0]

## test데이터
df_test['preferred_difficulty_level'] = pd.factorize(df_test['preferred_difficulty_level'])[0]
df_test['subscription_type'] = pd.factorize(df_test['subscription_type'])[0]

# scaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
for i in [['subscription_duration','recent_login_time','average_time_per_learning_session','monthly_active_learning_days','total_completed_courses','recent_learning_achievement','abandoned_learning_sessions','community_engagement_level','customer_inquiry_history','payment_pattern']]:
  df_train[i] = scaler.fit_transform(df_train[i])
for i in [['subscription_duration','recent_login_time','average_time_per_learning_session','monthly_active_learning_days','total_completed_courses','recent_learning_achievement','abandoned_learning_sessions','community_engagement_level','customer_inquiry_history','payment_pattern']]:
  df_test[i] = scaler.transform(df_test[i])

# Delete user_id
df_train = df_train.drop(columns = 'user_id')

# split target
x = df_train[list(df_train.columns[:-1])]
y = df_train['target']

# Delete user_id
df_test = df_test.drop(columns = 'user_id')
# split target
new_x = df_test


# import lib for modeling

In [3]:
!pip install catboost
!pip install optuna

In [6]:
import optuna
from lightgbm import LGBMClassifier,early_stopping
from catboost import CatBoostClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import make_scorer, f1_score
from sklearn.linear_model import LogisticRegression
import optuna
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier

## xgboost

In [7]:
def objective(trial, x, y):
  # Optuna가 탐색할 하이퍼파라미터의 범위 설정
  params = {
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
        'n_estimators': trial.suggest_int('n_estimators',1000,1001),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
        'subsample': trial.suggest_loguniform('subsample', 0.01, 1.0),
        'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.01, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 1.0),
        'eval_metric': 'logloss',
        'use_label_encoder': False
    }

  # k-겹 교차 검증 설정
  kf = KFold(
      n_splits =5,
      shuffle = True,
      random_state = 42)
  f1_scores = []

  for train_index, test_index in kf.split(x):
    x_train, x_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # xgBoost 모델생성
    model = XGBClassifier(**params, n_jobs = -1)
    model.fit(x_train, y_train, eval_set = [(x_test, y_test)], early_stopping_rounds = 100)

    # 예측 및 F 점수 계산
    preds = model.predict(x_test)
    f1 = f1_score(y_test, preds, average = 'macro')
    f1_scores.append(f1)

  return np.mean(f1_scores)

# Optuna  스터디 생성 및 최적화 실행
study = optuna.create_study(direction = 'maximize')
# lambda 함수를 사용하여 x와 y를 objective 함수에 전달
study.optimize(lambda trial: objective(trial, x, y), n_trials = 20)

# Optuna 스터디에서 최적의 하이퍼파라미터 가져오기
best_params = study.best_trial.params

# 최적의 하이퍼파라미터를 사용하여 CatBoost 모델 초기화
lgbm_model = LGBMClassifier(**best_params, verbose=0)

# Optuna 스터디에서 최적의 하이퍼파라미터 가져오기
best_params = study.best_trial.params

# 최적의 하이퍼파라미터를 사용하여 CatBoost 모델 초기화
xgb_model = XGBClassifier(**best_params, verbose=0)

[I 2023-12-06 11:11:26,646] A new study created in memory with name: no-name-f4b5a99b-15da-4a3e-8718-211faa902921


[0]	validation_0-logloss:0.66361
[1]	validation_0-logloss:0.66359
[2]	validation_0-logloss:0.66359
[3]	validation_0-logloss:0.66359
[4]	validation_0-logloss:0.66368
[5]	validation_0-logloss:0.66372
[6]	validation_0-logloss:0.66373
[7]	validation_0-logloss:0.66373
[8]	validation_0-logloss:0.66372
[9]	validation_0-logloss:0.66372
[10]	validation_0-logloss:0.66374
[11]	validation_0-logloss:0.66347
[12]	validation_0-logloss:0.66347
[13]	validation_0-logloss:0.66351
[14]	validation_0-logloss:0.66354
[15]	validation_0-logloss:0.66355
[16]	validation_0-logloss:0.66350
[17]	validation_0-logloss:0.66354
[18]	validation_0-logloss:0.66353
[19]	validation_0-logloss:0.66356
[20]	validation_0-logloss:0.66358
[21]	validation_0-logloss:0.66361
[22]	validation_0-logloss:0.66361
[23]	validation_0-logloss:0.66323
[24]	validation_0-logloss:0.66328
[25]	validation_0-logloss:0.66332
[26]	validation_0-logloss:0.66333
[27]	validation_0-logloss:0.66332
[28]	validation_0-logloss:0.66334
[29]	validation_0-loglos

[I 2023-12-06 11:12:12,270] Trial 0 finished with value: 0.38732106074736666 and parameters: {'max_depth': 6, 'learning_rate': 0.014139421184407896, 'n_estimators': 1000, 'min_child_weight': 5, 'gamma': 0.36653145567644285, 'subsample': 0.1682278935453077, 'colsample_bytree': 0.06591832654567586, 'reg_alpha': 0.00864089415937793, 'reg_lambda': 1.3782657926659546e-07}. Best is trial 0 with value: 0.38732106074736666.


[0]	validation_0-logloss:0.66406
[1]	validation_0-logloss:0.66194
[2]	validation_0-logloss:0.65977
[3]	validation_0-logloss:0.65829
[4]	validation_0-logloss:0.65936
[5]	validation_0-logloss:0.65915
[6]	validation_0-logloss:0.65856
[7]	validation_0-logloss:0.65788
[8]	validation_0-logloss:0.65936
[9]	validation_0-logloss:0.66070
[10]	validation_0-logloss:0.66030
[11]	validation_0-logloss:0.65929
[12]	validation_0-logloss:0.65868
[13]	validation_0-logloss:0.65887
[14]	validation_0-logloss:0.65955
[15]	validation_0-logloss:0.66067
[16]	validation_0-logloss:0.66125
[17]	validation_0-logloss:0.66062
[18]	validation_0-logloss:0.66260
[19]	validation_0-logloss:0.66311
[20]	validation_0-logloss:0.66290
[21]	validation_0-logloss:0.66332
[22]	validation_0-logloss:0.66293
[23]	validation_0-logloss:0.66354
[24]	validation_0-logloss:0.66357
[25]	validation_0-logloss:0.66485
[26]	validation_0-logloss:0.66423
[27]	validation_0-logloss:0.66314
[28]	validation_0-logloss:0.66409
[29]	validation_0-loglos

[I 2023-12-06 11:12:20,438] Trial 1 finished with value: 0.40349904222014166 and parameters: {'max_depth': 8, 'learning_rate': 0.08206921580533459, 'n_estimators': 1000, 'min_child_weight': 4, 'gamma': 0.001103230950741864, 'subsample': 0.09532445707782787, 'colsample_bytree': 0.5269621440523503, 'reg_alpha': 5.620612560629103e-07, 'reg_lambda': 0.030904045401884607}. Best is trial 1 with value: 0.40349904222014166.


...

[I 2023-12-06 11:18:34,052] Trial 17 finished with value: 0.39767761216816794 and parameters: {'max_depth': 7, 'learning_rate': 0.06459258741740757, 'n_estimators': 1001, 'min_child_weight': 2, 'gamma': 2.2928429989973137e-06, 'subsample': 0.04297967587557827, 'colsample_bytree': 0.9469420412227325, 'reg_alpha': 4.854810567989573e-05, 'reg_lambda': 1.100192774026945e-08}. Best is trial 13 with value: 0.4118214690562576.


[0]	validation_0-logloss:0.66438
[1]	validation_0-logloss:0.66305
[2]	validation_0-logloss:0.66191
[3]	validation_0-logloss:0.66145
[4]	validation_0-logloss:0.66185
[5]	validation_0-logloss:0.66210
[6]	validation_0-logloss:0.66253
[7]	validation_0-logloss:0.66214
[8]	validation_0-logloss:0.66186
[9]	validation_0-logloss:0.66218
[10]	validation_0-logloss:0.66093
[11]	validation_0-logloss:0.66083
[12]	validation_0-logloss:0.66139
[13]	validation_0-logloss:0.66121
[14]	validation_0-logloss:0.66186
[15]	validation_0-logloss:0.66158
[16]	validation_0-logloss:0.66140
[17]	validation_0-logloss:0.66098
[18]	validation_0-logloss:0.66148
[19]	validation_0-logloss:0.66182
[20]	validation_0-logloss:0.66292
[21]	validation_0-logloss:0.66265
[22]	validation_0-logloss:0.66363
[23]	validation_0-logloss:0.66348
[24]	validation_0-logloss:0.66395
[25]	validation_0-logloss:0.66415
[26]	validation_0-logloss:0.66456
[27]	validation_0-logloss:0.66437
[28]	validation_0-logloss:0.66519
[29]	validation_0-loglos

[I 2023-12-06 11:18:53,820] Trial 18 finished with value: 0.39046848052814936 and parameters: {'max_depth': 8, 'learning_rate': 0.04246254765770083, 'n_estimators': 1000, 'min_child_weight': 2, 'gamma': 1.6051310872422855e-05, 'subsample': 0.021323622943506336, 'colsample_bytree': 0.2409437007418278, 'reg_alpha': 8.820754565207746e-08, 'reg_lambda': 5.103739381056669e-06}. Best is trial 13 with value: 0.4118214690562576.


[0]	validation_0-logloss:0.66357
[1]	validation_0-logloss:0.66339
[2]	validation_0-logloss:0.66348
[3]	validation_0-logloss:0.66365
[4]	validation_0-logloss:0.66352
[5]	validation_0-logloss:0.66390
[6]	validation_0-logloss:0.66375
[7]	validation_0-logloss:0.66382
[8]	validation_0-logloss:0.66384
[9]	validation_0-logloss:0.66405
[10]	validation_0-logloss:0.66364
[11]	validation_0-logloss:0.66371
[12]	validation_0-logloss:0.66374
[13]	validation_0-logloss:0.66319
[14]	validation_0-logloss:0.66299
[15]	validation_0-logloss:0.66292
[16]	validation_0-logloss:0.66297
[17]	validation_0-logloss:0.66252
[18]	validation_0-logloss:0.66243
[19]	validation_0-logloss:0.66243
[20]	validation_0-logloss:0.66241
[21]	validation_0-logloss:0.66242
[22]	validation_0-logloss:0.66245
[23]	validation_0-logloss:0.66242
[24]	validation_0-logloss:0.66236
[25]	validation_0-logloss:0.66230
[26]	validation_0-logloss:0.66206
[27]	validation_0-logloss:0.66194
[28]	validation_0-logloss:0.66194
[29]	validation_0-loglos

[I 2023-12-06 11:19:04,561] Trial 19 finished with value: 0.3828884002256786 and parameters: {'max_depth': 7, 'learning_rate': 0.04697637315109997, 'n_estimators': 1000, 'min_child_weight': 7, 'gamma': 9.47604713616952e-07, 'subsample': 0.010399759927172882, 'colsample_bytree': 0.4271154456593954, 'reg_alpha': 1.1554299951593803e-06, 'reg_lambda': 1.726883491360272e-05}. Best is trial 13 with value: 0.4118214690562576.


## light Gbm

In [ ]:
def objective(trial, x, y):
  # Optuna가 탐색할 하이퍼파라미터의 범위 설정
  params = {
        'objective': 'binary',
        'num_leaves': trial.suggest_int('num_leaves', 100, 500, step=1, log=True),
        'max_depth': trial.suggest_int('max_depth', 1, 10, step=1, log=False),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 900, 1000, step=1, log=True),
        'class_weight': trial.suggest_categorical('class_weight', ['balanced', None]),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 50, step=10, log=False),
        'subsample': trial.suggest_uniform('subsample', 0.7, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.7, 1.0),
        'reg_alpha': trial.suggest_uniform('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_uniform('reg_lambda', 0.0, 10.0),
        'random_state': 0
  }

  # k-겹 교차 검증 설정
  kf = KFold(
      n_splits =10,
      shuffle = True,
      random_state = 42)
  f1_scores = []

  for train_index, test_index in kf.split(x):
    x_train, x_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # xgBoost 모델생성
    model = LGBMClassifier(**params, n_jobs = -1)
    model.fit(x_train, y_train, eval_set = [(x_test, y_test)],callbacks=[early_stopping(stopping_rounds=100)], eval_metric= 'logloss')

    # 예측 및 F 점수 계산
    preds = model.predict(x_test)
    f1 = f1_score(y_test, preds, average = 'macro')
    f1_scores.append(f1)

  return np.mean(f1_scores)

# Optuna  스터디 생성 및 최적화 실행
study = optuna.create_study(direction = 'maximize')
# lambda 함수를 사용하여 x와 y를 objective 함수에 전달
study.optimize(lambda trial: objective(trial, x, y), n_trials = 40)

# Optuna 스터디에서 최적의 하이퍼파라미터 가져오기
best_params = study.best_trial.params

# 최적의 하이퍼파라미터를 사용하여 CatBoost 모델 초기화
lgbm_model = LGBMClassifier(**best_params, verbose=0)


# logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score


# 목적 함수 정의
def obj_log(trial,x,y):
    C = trial.suggest_float('C', 1e-4, 1e4, log=True)
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])

    # 솔버 선택
    if penalty == 'l1':
        solver = 'liblinear'
    else:
        solver = trial.suggest_categorical('solver', ['lbfgs', 'newton-cg', 'sag'])

    model = LogisticRegression(C=C, penalty=penalty, solver=solver, max_iter=1000)
    score = cross_val_score(model, x, y, n_jobs=-1, cv=3, scoring='f1_macro')
    f1_macro = np.mean(score)

    return f1_macro

# Optuna 최적화 실행
study = optuna.create_study(direction='maximize')
# lambda 함수를 사용하여 x와 y를 objective 함수에 전달
study.optimize(lambda trial: obj_log(trial, x, y), n_trials = 60)

# 최적 파라미터 출력
print("Best trial:")
trial = study.best_trial
print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

# 최적 파라미터로 모델 학습
best_params = study.best_trial.params
logi_model = LogisticRegression(**best_params, max_iter=1000)


## knn

In [ ]:
# 목적 함수 정의
def obj_knn(trial,x,y):
    n_neighbors = trial.suggest_int('n_neighbors', 1, 30)
    metric = trial.suggest_categorical('metric', ['euclidean', 'manhattan', 'minkowski'])
    weights = trial.suggest_categorical('weights', ['uniform', 'distance'])

    model = KNeighborsClassifier(n_neighbors=n_neighbors, metric=metric, weights=weights)
    score = cross_val_score(model, x, y, n_jobs=-1, cv=3, scoring='f1_macro')
    f1_macro = np.mean(score)

    return f1_macro

# Optuna 최적화 실행
study = optuna.create_study(direction='maximize')
# lambda 함수를 사용하여 x와 y를 objective 함수에 전달
study.optimize(lambda trial: obj_knn(trial, x, y), n_trials = 60)


# 최적 파라미터 출력
print("Best trial:")
trial = study.best_trial
print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

# 최적 파라미터로 모델 학습
best_params = study.best_trial.params
knn_model = KNeighborsClassifier(**best_params)

# CV(CrossValidation)기반 stacking

In [ ]:
def stacking_run(model, x_train, y_train, x_test, n_folds):
  kf = KFold(n_splits = n_folds, shuffle = False, random_state = 0)

  # 추후 메타 모델이 사용할 학습 데이터 반환을 위한 넘파이 배열 초기화
  train_fold_pred = np.zeros((x_train.shape[0],1))
  test_pred = np.zeros((x_test.shape[0], n_folds))

  print(model.__class__.__name__,'model 시작')

  for folder_counter, (train_index, valid_index)

In [ ]:
import optuna
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score

# 사용할 모델 선정
# 1. lightgbm
# 2. catboost
# 3. logistic
# 4. SVM
# 5. KNN
# 6. 나이브베이즈



# catboost 하이퍼파라미터
cat_params= {
    'iterations': trial.suggest_int('iterations', 800, 1000),
    'depth': trial.suggest_int('depth', 4, 10),
    'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5),
    'random_strength': trial.suggest_int('random_strength', 0, 100),
    'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
    'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter'])
}


# LightGbm 하이퍼파라미터
params = {
      'objective': 'binary',
      'num_leaves': trial.suggest_int('num_leaves', 100, 500, step=1, log=True),
      'max_depth': trial.suggest_int('max_depth', 1, 10, step=1, log=False),
      'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
      'n_estimators': trial.suggest_int('n_estimators', 900, 1000, step=1, log=True),
      'class_weight': trial.suggest_categorical('class_weight', ['balanced', None]),
      'min_child_samples': trial.suggest_int('min_child_samples', 10, 50, step=10, log=False),
      'subsample': trial.suggest_uniform('subsample', 0.7, 1.0),
      'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.7, 1.0),
      'reg_alpha': trial.suggest_uniform('reg_alpha', 0.0, 1.0),
      'reg_lambda': trial.suggest_uniform('reg_lambda', 0.0, 10.0),
      'random_state': 0
}

# xgboost 하이퍼파라미터
params = {
      "objective":'binary:logistic',
      'max_depth': trial.suggest_int('max_depth', 3, 9),
      'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
      'n_estimators': trial.suggest_int('n_estimators',1000,1001),
      'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
      'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
      'subsample': trial.suggest_loguniform('subsample', 0.01, 1.0),
      'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.01, 1.0),
      'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
      'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 1.0),
      'eval_metric': 'logloss',
      'use_label_encoder': False
  }




